In [7]:
import csv
import tqdm

In [69]:
def list_to_str(list):
    res = ''
    for item in list:
        res = res + item
        res = res + ' '
    return res.rstrip()

In [70]:
def read_file(filename):
    with open(filename) as f:
        reader = csv.DictReader(f)
        message = 1
        
        head = next(reader)
        print(head.keys())
        pre_time = head['timestamp']
        pre_id = head['user_id']
        pre_member = head['members']
        
        head_mention = head['mentions'].rstrip().split(' ')
        pre_mentions = set(head_mention) 
        
        info_list = list()
        for row in reader:
            time = row['timestamp']
            user_id = row['user_id']
            member = row['members']
            mentions = row['mentions']
            
            # transform mentions into list and then into set
            mentions_set = set(mentions.rstrip().split(' '))
            if pre_time == time and pre_id == user_id and pre_member == member:
                message += 1
                pre_mentions = pre_mentions.union(mentions_set)
                
                # if new mentions come in, then delete '0'
                if '0' in pre_mentions and len(pre_mentions) > 1:
                    pre_mentions.remove('0')
                
            else: 
                # transfrom mentions set to string
                pre_mentions_list = list(pre_mentions)
                pre_mentions_str = " ".join(str(st) for st in pre_mentions_list)
                
                # add pre information into a list and then append into the total list
                sub_list = [pre_time, pre_id, pre_member, pre_mentions_str, message]
                info_list.append(sub_list)
                
                # reset pre-variables to the one just read
                message = 1
                pre_time, pre_id, pre_member = time, user_id, member
                pre_mentions = mentions_set
        
        # add the last list into the total list
        pre_mentions_list = list(pre_mentions)
        pre_mentions_str = " ".join(st for st in pre_mentions_list)
        sub_list = [pre_time, pre_id, pre_member, pre_mentions_str, message]
        
        
        info_list.append(sub_list)
        return info_list

In [94]:
def read_file_processed(filename):
    with open(filename) as f:
        reader = csv.DictReader(f)
        head = next(reader)
        
        # collect timestamp and user id
        pre_time = head['timestamp']
        pre_id = head['user_id']
        
        #collect members into a list
        pre_members = set(head['members'].rstrip().split(' '))
        
        #collect mentions into a list
        pre_mentions = head['mentions'].rstrip().split(' ')
        
        #collect message into a list
        pre_message = list()
        pre_message.append(head['number_message'])

        
        #define final result
        info_list = list()
        max_len = len(pre_members)
        
        for row in reader:
            time = row['timestamp']
            user_id = row['user_id']
            member = row['members']
            mentions = row['mentions']
            message = row['number_message']
            
            if pre_time == time and pre_id == user_id:
                pre_members = pre_members.union(set(member.rstrip().split(' ')))
                pre_mentions += mentions.rstrip().split(' ')
                pre_message.append(message)
                if(len(pre_members) > max_len):
                    max_len = len(pre_members)
            
            else:
                sub_list = [pre_time, pre_id, list_to_str(pre_members), list_to_str(pre_mentions), list_to_str(pre_message)]
                info_list.append(sub_list)
                pre_time, pre_id, pre_members, pre_mentions = time, user_id, set(member.rstrip().split(' ')), mentions.rstrip().split(' ')
                pre_message = list()
                pre_message.append(message)
        
        
        sub_list = [pre_time, pre_id, list_to_str(list(pre_members)), list_to_str(pre_mentions), list_to_str(pre_message)]
        info_list.append(sub_list)
        return info_list, max_len

In [95]:
def write_process(filename, info_list):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        for sub_list in tqdm.tqdm(info_list):
            writer.writerow(sub_list)
        print('Write Finished')   

In [102]:
def main(args):
    # step 1, conclude the original information to count message mentions and members
    if args == 'Form Processed Slack':
        info_list = read_file_pre_processed('data/Slack_Private_19062018.csv')
        write_process('data/Slack_Private_Processed.csv', info_list)
    
    # step 2, conclude the final infomation with the key of id and date
    elif args == 'Form Final Slack':
        info_list, max_len = read_file_processed('data/Slack_Private_Processed.csv')
        #for sub_list in info_list:
        #       print(sub_list)
        write_process('data/Slack_Private_Final.csv', info_list)
        print(max_len)

In [103]:
if __name__ == '__main__':
    args = 'Form Final Slack'

    main(args)

100%|██████████| 11405/11405 [00:00<00:00, 505971.22it/s]

Write Finished
26
